In [9]:
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib
import webbrowser
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time

In [10]:
headers = {
    """
    En-têtes HTTP couramment utilisés pour simuler une requête HTTP provenant d'un navigateur. Ces en-têtes peuvent être ajustés en fonction des besoins spécifiques de la requête.

    - 'User-Agent': Identifie le type de navigateur et le système d'exploitation utilisé.
    - 'Accept': Indique les types de contenu que le client peut traiter.
    - 'Accept-Language': Spécifie les langues préférées pour la réponse.
    - 'Connection': Contrôle si la connexion réseau doit être maintenue ou non après la transaction courante.
    """
    
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_1_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=1.0,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr;q=0.9',
    'Connection': 'keep-alive',
}

## ETAPE 1 : ACQUISITION DES DONNEES

In [11]:
def extract_autorite(csv_path, column_name_1) :
    """
    Extrait les url de Phoebus des notices d'autorité ainsi que la Source (uri d'IdRef)

    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données.
        column_name_1 (str): colonne à partir de laquelle extraire les urls de recherche sur Phoebus.
        column_name_2 (str) : colonne à contenant l'uri d'IdRef

    """
    # charger les données csv
    df = pd.read_csv(csv_path, delimiter=',') #delimiter dépend de la configuration du separator du csv
    # Extraire les noms de la colonne
    names_to_extract = df[column_name_1].tolist()
    return names_to_extract

def extract_idRef(csv_path, column_name_2) :
    """
    Extrait les url d'Idref des notices d'autorité et les modifie

    Args:
        csv_path (str): Chemin vers le fichier CSV contenant les données.
        column_name_2 (str): Nom de la colonne à partir de laquelle extraire les noms.

    Returns:
        list: Liste des noms extraits de la colonne spécifiée.
    """
    # charger les données csv
    df = pd.read_csv(csv_path, delimiter=',') #delimiter dépend de la configuration du separator du csv
    # Extraire les noms de la colonne
    idRef_links = df[column_name_2].tolist()
    
    # Convertir les valeurs float en chaînes de caractères
    idRef_links = [str(link) for link in idRef_links]
    
    idRef_links_xml = [link.replace('/id', '.xml') for link in idRef_links]    
    return idRef_links_xml

# LANCEMENT DE LA FONCTION
csv_path = '01_autorite_Id.csv'
column_name_1 = 'url_phoebus'
column_name_2 = 'Sources'
names_to_extract = extract_autorite(csv_path, column_name_1)
idRef_links = extract_idRef(csv_path, column_name_2)

# Afficher les noms (à titre de vérification)
print(names_to_extract)
print(idRef_links)

['https://atom-archives.unil.ch/index.php/aerny-francis', 'https://atom-archives.unil.ch/index.php/aleixandre-vicente', 'https://atom-archives.unil.ch/index.php/allini-pier', 'https://atom-archives.unil.ch/index.php/andres-stefan']
['https://www.idref.fr/263915085.xml', 'https://www.idref.fr/026648261.xml', 'https://www.idref.fr/273130536.xml', 'https://www.idref.fr/072274670.xml']


## ETAPE 2 : RECUPERATION DES DONNEES SUR IDREF AVEC GESTION DES ERREURS DUES AU TIME OUT

In [12]:
def extract_from_idRef(xml_urls, timeout_value):
    """
    Extrait les données à partir d'une liste d'URLs pointant vers des enregistrements XML sur le service IDRef.

    Args:
        xml_urls (list): Liste d'URLs pointant vers des enregistrements XML sur IDRef.
        timeout_value (int): Durée maximale lors de la tentative de connexion.

    Returns:
        list: Liste d'enregistrements extraits contenant les données récupérées depuis les enregistrements XML.
              - 'ErrorTimeOut' : Erreur due à un dépassement du délai de connexion.
    """
    
    
    # GESTION DES TIME OUT ERROR : ESSAIS DE CONNEXION
    # Configuration des tentatives de reconnexion
    retries = Retry(total=3, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
    # Création d'un adaptateur avec les tentatives de reconnexion configurées
    adapter = HTTPAdapter(max_retries=retries)
    
    
    extracted_from_idRef = []
    # Création d'une session avec l'adaptateur
    with requests.Session() as session:
        session.mount('https://', adapter)
    
        for i, xml_url in enumerate(xml_urls):

            # Initialiser une liste pour cette notice
            notice = []
            # Initialiser les variables pour cet enregistrement
            authorizedFormOfName = ""
            nomAutorise = ""
            prenomAutorise = ""
            dateExistence = ""
            functions = ""
            infoPublique =""
            biographies = ""
            history =""
            URI = ""
            tous_AutresNomsPrenoms = ""
            identifiant = ""

            try :    
                 # Récupérer le contenu XML depuis l'URL
                response = session.get(xml_url, timeout = timeout_value)
                response.raise_for_status()
                xml_content = response.text
                # Ajouter un délai entre les requêtes
                time.sleep(5)

                # Charger le contenu XML avec BeautifulSoup
                soup = BeautifulSoup(xml_content, 'xml')

                # FORME AUTORISEE DE NOM : Parcourir toutes les balises <datafield> avec les tags spécifiés (200)
                tag200_datafields = soup.find('datafield', {'tag': ['200']})
                if tag200_datafields :
                    # Le tag 200 existe alors extraire le NOM, Prénom, les années d'existence
                    nomAutorise = tag200_datafields.find('subfield', {'code': 'a'}).text if tag200_datafields.find('subfield', {'code': 'a'}) else ''
                    prenomAutorise = tag200_datafields.find('subfield', {'code': 'b'}).text if tag200_datafields.find('subfield', {'code': 'b'}) else ''
                    dateExistence = tag200_datafields.find('subfield', {'code': 'f'}).text if tag200_datafields.find('subfield', {'code': 'f'}) else 's.d.'
                    functions = tag200_datafields.find('subfield', {'code': 'c'}).text if tag200_datafields.find('subfield', {'code': 'c'}) else 'NULL'

                    # Mettre en forme les noms autorisées
                    nomAutorise = nomAutorise.upper()
                    # Concaténer les valeurs du nom autorisé
                    authorizedFormOfName = f'{nomAutorise} {prenomAutorise}'
                else:
                    # Le tag 200 n'existe pas
                    nomAutorise = "NULL"
                    prenomAutorise = "NULL"
                    dateExistence = "NULL"
                    functions = "NULL"
                    
                # Note publique d'information
                tag300_datafields = soup.find_all('datafield', {'tag':'300'})

                if tag300_datafields:
                    for tag300_datafield in tag300_datafields :
                        infoPublique = tag300_datafield.find('subfield', {'code': 'a'}).text if tag300_datafield.find('subfield', {'code': 'a'}) else ''
                else :
                    infoPublique = "NULL"
                        
                # Ajouter un point à la fin
                infoPublique += f"."
                # Supprimer les doubles espaces
                infoPublique = infoPublique.replace("  "," ")
                # Remplacer " ; ." par "." pour terminer la liste
                infoPublique = infoPublique.replace(" ; .",".")
                # Remplacer "NULL." par "NULL" pour uniformiser toutes les valeurs NULL
                infoPublique = infoPublique.replace("NULL.","NULL")
                # supprimer les espaces avant et à la fin
                infoPublique = infoPublique.strip()        

                # HISTORIQUE : Parcourir toutes les balises <datafield> avec les tags spécifiés (340)
                tag340_datafields = soup.find_all('datafield', {'tag':'340'})

                if tag340_datafields:
                    for tag340_datafield in tag340_datafields :
                        biographie = tag340_datafield.find('subfield', {'code': 'a'}).text if tag340_datafield.find('subfield', {'code': 'a'}) else ''
                        biographie = f"{biographie} ; "
                        biographies += biographie
                else:
                # Si aucune biographie trouvée, définir comme "NULL"
                    biographies ="NULL"

                # Ajouter un point à la fin
                biographies += f"."
                # Supprimer les doubles espaces
                biographies = biographies.replace("  "," ")
                # Remplacer " ; ." par "." pour terminer la liste
                biographies = biographies.replace(" ; .",".")
                # Remplacer "NULL." par "NULL" pour uniformiser toutes les valeurs NULL
                biographies = biographies.replace("NULL.","NULL")
                # supprimer les espaces avant et à la fin
                biographies = biographies.strip()

                #Mise en forme history en corrigeant la concaténation de biographies et infoPublique
                history = f"{biographies} ; {infoPublique}"
                history = history.replace("..",".")
                history = history.replace("NULL ; NULL","NULL")
                history = history.replace("NULL ; ","")
                history = history.replace("; NULL","")

                 # SOURCES : Extraire le lien URI
                URI = soup.find('controlfield', {'tag': '003'}).text if soup.find('controlfield', {'tag': '003'}) else ''
                # DATE DE CREATION de la notice : 
                creationDate = soup.find('controlfield', {'tag': '004'}).text if soup.find('controlfield', {'tag': '004'}) else ''
                # DATE DE MODIFICATION de la notice : 
                modificationDate = soup.find('controlfield', {'tag': '005'}).text if soup.find('controlfield', {'tag': '005'}) else ''

                #METTRE EN FORME URI, Dates de création et de modification)
                URI = URI + "/id"
                # Formater la date de création
                creationDate = datetime.strptime(creationDate, "%Y%m%d").strftime("%Y-%m-%d")
                # Formater la date de modification
                modificationDate = datetime.strptime(modificationDate, "%Y%m%d%H%M%S.%f").strftime("%Y-%m-%d à %H:%M:%S")
                #METTRE EN FORME LE CHAMP SOURCE
                referenceIdRef = f"Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur {URI}"
                #METTRE EN FORME LE CHAMP DATES DE PROD, REVISION
                revisionHistory = f"Notice ABES. Date de création : {creationDate} - Date de modification : {modificationDate}."

                # AUTRES FORMES DE NOM : Parcourir TOUTES les balises <datafield> avec le tag "400" //dans la structure XML, il y en a plusieurs 
                tag400_datafields = soup.find_all('datafield', {'tag': ['400']})
                if tag400_datafields :
                    # Au moins une occurrence du tag 400 existe
                    for tag400_datafield in tag400_datafields :
                        # Extraire les sous-champs 'a' et 'b'
                        autreNom = tag400_datafield.find('subfield', {'code': 'a'}).text if tag400_datafield.find('subfield', {'code': 'a'}) else ''
                        autrePrenom = tag400_datafield.find('subfield', {'code': 'b'}).text if tag400_datafield.find('subfield', {'code': 'b'}) else ''

                        # Mettre en forme et concaténer les autres Noms
                        autreNom = autreNom.upper()
                        autresNomsPrenoms = f"{autreNom} {autrePrenom} ; "
                        # Concaténer les valeurs des noms alternatifs
                        tous_AutresNomsPrenoms += autresNomsPrenoms
                else:
                    # Le tag 400 n'existe pas
                    tous_AutresNomsPrenoms = "NULL"

                # Ajouter un point à la fin
                tous_AutresNomsPrenoms += f"."
                # Supprimer les doubles espaces
                tous_AutresNomsPrenoms = tous_AutresNomsPrenoms.replace("  "," ")
                # Remplacer " ; ." par "." pour terminer la liste des autres formes de nom
                tous_AutresNomsPrenoms = tous_AutresNomsPrenoms.replace(" ; .",".")
                # Remplacer "NULL." par "NULL" pour uniformiser toutes les valeurs NULL
                tous_AutresNomsPrenoms = tous_AutresNomsPrenoms.replace("NULL.","NULL")
                # supprimer les espaces avant et à la fin
                tous_AutresNomsPrenoms = tous_AutresNomsPrenoms.strip()
                
                # IDENTIFIANTS : prendre l'identifiant idref
                identifiant = soup.find('controlfield', {'tag': '001'}).text if soup.find('controlfield', {'tag': '003'}) else 'ERROR'
                identifiant = "idref-"+identifiant
                

                # Ajouter les données (si données existent ou non) à la liste "notice"
                notice.append(authorizedFormOfName)
                notice.append(dateExistence)
                notice.append(functions)
                notice.append(history)
                notice.append(revisionHistory)
                notice.append(referenceIdRef)
                notice.append(tous_AutresNomsPrenoms)
                notice.append(identifiant)

                # Ajouter l'enregistrement à la liste principale
                extracted_from_idRef.append(notice)

            # GESTION DES ERREURS DU AU TIME OUT    
            except (requests.Timeout, requests.RequestException) as e:
                notice.extend(["ErrorTimeOut"]*5)
                notice.append(xml_url)
                notice.append("ErrorTimeOut")
                # Ajouter l'enregistrement à la liste principale
                extracted_from_idRef.append(notice)
                print(f"Erreur de time out lors de la requête pour {names[i]} : {xml_url}")
                print(f"Exception : {e}")

            #except Exception as e:
                print(f"Erreur non gérée lors de la requête pour {names[i]} : {xml_url}. Erreur : {type(e).__name__}")


    return extracted_from_idRef
            

# LANCEMENT DE LA FONCTION
xml_urls = idRef_links
names = names_to_extract
timeout_value = 10 # détermine la durée maximale lors de la tentative de connexion.
extracted_from_idRef = extract_from_idRef(xml_urls, timeout_value)

print(extracted_from_idRef)

[['AERNY Francis', '19..-....', 'NULL', 'Professeur. ', 'Notice ABES. Date de création : 2022-08-23 - Date de modification : 2024-06-26 à 15:02:53.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur http://www.idref.fr/263915085/id", 'NULL', 'idref-263915085'], ['ALEIXANDRE Vicente', '1898-1984', 'NULL', 'Poète espagnol.', 'Notice ABES. Date de création : 1984-11-09 - Date de modification : 2024-06-26 à 15:00:55.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur http://www.idref.fr/026648261/id", 'ALEIXANDRE Y MERLO Vicente ; MERLO Vicente Aleixandre y.', 'idref-026648261'], ['ALLINI Pier', 's.d.', 'NULL', 'NULL', 'Notice ABES. Date de création : 2023-11-14 - Date de modification : 2024-06-26 à 15:02:03.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur http://www.idref.fr/273130536/id", 'NULL', 'idref-273130536'], ['ANDRES Stefan Paul', '1906-1970

## ETAPE 3 : MISE EN FORME DES DONNEES

In [13]:
def format_data(list1, list2):
    """
    Combinaison de deux listes en une liste de listes.

    Args:
        list1 (list): Liste contenant les urls tirés de Phoebus à utiliser comme premiers éléments de chaque sous-liste.
        list2 (list): Liste contenant les données récupérées d'IDRef à utiliser comme éléments suivants de chaque sous-liste.

    Returns:
        list: Liste de listes où chaque sous-liste commence par un élément de list1 suivi des éléments correspondants de list2.

    """
    # Reformater les données en une liste de listes
    formatted_data = []

    for name, idRef_info in zip(list1, list2):
                
        #Modifier l'identifiant en ajoutant l'url 
        url_parts = name.split("/")
        identifier = url_parts[-1]  # Récupérer la dernière partie de l'URL
        
        idRef_info[-1] = identifier + "_" + idRef_info[-1]
        formatted_data.append([name] + idRef_info)

    return formatted_data

# LANCEMENT DE LA FONCTION
# Données à écrire dans le csv
list1 = names_to_extract
list2 = extracted_from_idRef
formatted_data = format_data(list1, list2)
print(formatted_data)

[['https://atom-archives.unil.ch/index.php/aerny-francis', 'AERNY Francis', '19..-....', 'NULL', 'Professeur. ', 'Notice ABES. Date de création : 2022-08-23 - Date de modification : 2024-06-26 à 15:02:53.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur http://www.idref.fr/263915085/id", 'NULL', 'aerny-francis_idref-263915085'], ['https://atom-archives.unil.ch/index.php/aleixandre-vicente', 'ALEIXANDRE Vicente', '1898-1984', 'NULL', 'Poète espagnol.', 'Notice ABES. Date de création : 1984-11-09 - Date de modification : 2024-06-26 à 15:00:55.', "Agence bibliographique de l'Enseignement supérieur (ABES) - Données originales récupérées sur http://www.idref.fr/026648261/id", 'ALEIXANDRE Y MERLO Vicente ; MERLO Vicente Aleixandre y.', 'aleixandre-vicente_idref-026648261'], ['https://atom-archives.unil.ch/index.php/allini-pier', 'ALLINI Pier', 's.d.', 'NULL', 'NULL', 'Notice ABES. Date de création : 2023-11-14 - Date de modification : 2024-06-2

## ETAPE 4 : ECRITURE DES DONNEES DANS LE FICHIER CSV

In [14]:
def write_to_csv(csv_filename, headers, formatted_data) :
    """
    Écrit les données dans un fichier CSV.

    Args:
        csv_filename (str): Nom du fichier CSV à créer ou mettre à jour.
        headers (list): Liste des en-têtes de colonnes pour le fichier CSV.
        formatted_data (list): Liste de listes contenant les données à écrire dans le fichier CSV.

    Returns:
        None
    """    
    # Écriture dans le fichier CSV
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter=',')

         # Écrire l'en-tête du CSV
        csv_writer.writerow(headers)
        
        # Écrire les données dans le fichier CSV
        csv_writer.writerows(formatted_data)

# LANCEMENT DE LA FONCTION
csv_filename = '02_resultats.csv'
headers = ['url_phoebus', 'Forme autorisée de nom', 'DateExistence', 'Activités','Historique', 'Historique de révision','Sources', 'Autre(s) forme(s) du nom','Identifiant']

write_to_csv(csv_filename, headers, formatted_data)

## LANCEMENT DES FONCTIONS

In [ ]:
# ETAPE 1
csv_path = '01_autorite_Id.csv'
column_name_1 = 'url_phoebus'
column_name_2 = 'Sources'
names_to_extract = extract_autorite(csv_path, column_name_1)
idRef_links = extract_idRef(csv_path, column_name_2)

# ETAPE 2
xml_urls = idRef_links
names = names_to_extract
timeout_value = 10 # détermine la durée maximale lors de la tentative de connexion.
extracted_from_idRef = extract_from_idRef(xml_urls, timeout_value)

# ETAPE 3
list1 = names_to_extract
list2 = extracted_from_idRef
formatted_data = format_data(list1, list2)

# ETAPE 4
csv_filename = '02_resultats.csv'
headers = ['url_phoebus', 'Forme autorisée de nom', 'DateExistence', 'Activités','Historique', 'Historique de révision','Sources', 'Autre(s) forme(s) du nom','Identifiant']
write_to_csv(csv_filename, headers, formatted_data)
